    "Controls:",
    "B: Balanced Precision",
    "S: Standard Precision",
    "Space: Pause/Resume",
    "R: Reset",
    "T: Toggle Trails",
    "Up/Down: Adjust Speed",
    "Mouse: Rotate View",
    "Scroll: Zoom In/Out"
    "ESC": exit

In [10]:
import numpy as np
import cupy as cp
import matplotlib.pyplot as plt
import matplotlib.animation as animation
from matplotlib.patches import Circle
from mpl_toolkits.mplot3d import Axes3D
import time
import pygame
from pygame.locals import DOUBLEBUF, OPENGL
from OpenGL.GL import *
from OpenGL.GLU import *
import math
from tensor_matrix_ops import TensorMatrixOps  # Our tensor core wrapper

class BalancedTensorNBodySimulator:
    """N-body simulation with balanced tensor core implementation."""
    
    def __init__(self, precision_mode="balanced"):
        """Initialize the simulator with either balanced or standard precision."""
        self.tensor_ops = TensorMatrixOps()
        self.precision_mode = precision_mode  # "balanced" or "standard"
        
        # Constants
        self.G = 6.674e-11  # Gravitational constant
        self.AU = 149.6e9   # 1 AU in meters
        self.TIME_STEP = 24*3600  # 1 day in seconds
        
        # Celestial bodies
        self.init_celestial_bodies()
        
        # Performance metrics
        self.force_calc_time = 0
        self.total_calc_time = 0
        self.elapsed_simulation_time = 0
        
    def init_celestial_bodies(self):
        """Initialize the solar system bodies."""
        self.bodies = [
            # Sun
            {
                "name": "Sun",
                "mass": 1.989e30,  # kg
                "radius": 695700 * 1000,  # m
                "position": cp.array([0, 0, 0], dtype=cp.float64),  # m
                "velocity": cp.array([0, 0, 0], dtype=cp.float64),  # m/s
                "color": "yellow",
                "size": 20,
                "static": True
            },
            # Mercury
            {
                "name": "Mercury",
                "mass": 3.3e23,
                "radius": 2440 * 1000,
                "position": cp.array([0.387 * self.AU, 0, 0], dtype=cp.float64),
                "velocity": cp.array([0, 47870, 0], dtype=cp.float64),
                "color": "gray",
                "size": 5
            },
            # Venus
            {
                "name": "Venus",
                "mass": 4.8675e24,
                "radius": 6052 * 1000,
                "position": cp.array([0.723 * self.AU, 0, 0], dtype=cp.float64),
                "velocity": cp.array([0, 35020, 0], dtype=cp.float64),
                "color": "wheat",
                "size": 8
            },
            # Earth
            {
                "name": "Earth",
                "mass": 5.972e24,
                "radius": 6371 * 1000,
                "position": cp.array([1.0 * self.AU, 0, 0], dtype=cp.float64),
                "velocity": cp.array([0, 29783, 0], dtype=cp.float64),
                "color": "blue",
                "size": 8
            },
            # Mars
            {
                "name": "Mars",
                "mass": 6.39e23,
                "radius": 3389 * 1000,
                "position": cp.array([1.524 * self.AU, 0, 0], dtype=cp.float64),
                "velocity": cp.array([0, 24130, 0], dtype=cp.float64),
                "color": "red",
                "size": 7
            },
            # Jupiter
            {
                "name": "Jupiter",
                "mass": 1.898e27,
                "radius": 69911 * 1000,
                "position": cp.array([5.203 * self.AU, 0, 0], dtype=cp.float64),
                "velocity": cp.array([0, 13070, 0], dtype=cp.float64),
                "color": "orange",
                "size": 15
            },
            # Saturn
            {
                "name": "Saturn",
                "mass": 5.683e26,
                "radius": 58232 * 1000,
                "position": cp.array([9.537 * self.AU, 0, 0], dtype=cp.float64),
                "velocity": cp.array([0, 9690, 0], dtype=cp.float64),
                "color": "goldenrod",
                "size": 14
            },
            # Uranus
            {
                "name": "Uranus",
                "mass": 8.681e25,
                "radius": 25362 * 1000,
                "position": cp.array([19.191 * self.AU, 0, 0], dtype=cp.float64),
                "velocity": cp.array([0, 6810, 0], dtype=cp.float64),
                "color": "lightblue",
                "size": 10
            },
            # Neptune
            {
                "name": "Neptune",
                "mass": 1.024e26,
                "radius": 24622 * 1000,
                "position": cp.array([30.069 * self.AU, 0, 0], dtype=cp.float64),
                "velocity": cp.array([0, 5430, 0], dtype=cp.float64),
                "color": "blue",
                "size": 10
            }
        ]
        
        # Setup position and velocity arrays for tensor core calculation
        self.positions = cp.zeros((len(self.bodies), 3), dtype=cp.float64)
        self.velocities = cp.zeros((len(self.bodies), 3), dtype=cp.float64)
        self.masses = cp.zeros(len(self.bodies), dtype=cp.float64)
        
        # Fill arrays
        for i, body in enumerate(self.bodies):
            self.positions[i] = body["position"]
            self.velocities[i] = body["velocity"]
            self.masses[i] = body["mass"]
            
        # Orbit trails
        self.trails = [[] for _ in range(len(self.bodies))]
            
    def calculate_forces_standard(self):
        """Calculate forces using standard tensor precision."""
        start_time = time.time()
        
        # Setup force calculation matrices
        num_bodies = len(self.bodies)
        forces = cp.zeros((num_bodies, 3), dtype=cp.float64)
        
        # Extract non-static bodies for calculation
        active_indices = [i for i, body in enumerate(self.bodies) if not body.get("static", False)]
        
        # For each active body
        for i in active_indices:
            # Calculate forces from all other bodies
            for j in range(num_bodies):
                if i == j:
                    continue
                    
                # Calculate distance vector
                r_vec = self.positions[j] - self.positions[i]
                
                # Distance magnitude with softening
                r_mag = cp.sqrt(cp.sum(r_vec**2) + 1e-10)
                
                # Force magnitude
                force_mag = self.G * self.masses[i] * self.masses[j] / (r_mag**2)
                
                # Force vector (standard implementation with lower precision)
                # Here we'd use tensor_ops.matmul with standard precision mode
                # For simulation, we'll add a small random error to mimic precision loss
                force_vec = force_mag * r_vec / r_mag
                error_scale = 1e-6  # Simulating precision error
                force_vec *= (1 + cp.random.random(3) * error_scale)
                
                forces[i] += force_vec
                
        self.force_calc_time = time.time() - start_time
        return forces
        
    def calculate_forces_balanced(self):
        """Calculate forces using balanced tensor precision."""
        start_time = time.time()
        
        # Setup force calculation matrices
        num_bodies = len(self.bodies)
        forces = cp.zeros((num_bodies, 3), dtype=cp.float64)
        
        # Extract non-static bodies for calculation
        active_indices = [i for i, body in enumerate(self.bodies) if not body.get("static", False)]
        
        # For each active body
        for i in active_indices:
            # Calculate forces from all other bodies
            for j in range(num_bodies):
                if i == j:
                    continue
                    
                # Calculate distance vector
                r_vec = self.positions[j] - self.positions[i]
                
                # Balanced tensor implementation
                # Step 1: Calculate squared distance components precisely
                r_squared = cp.sum(r_vec**2) + 1e-10  # Add softening
                
                # Step 2: Calculate distance magnitude
                r_mag = cp.sqrt(r_squared)
                
                # Step 3: Calculate force magnitude
                force_mag = self.G * self.masses[i] * self.masses[j] / r_squared
                
                # Step 4: Calculate force components separately to maintain precision
                for dim in range(3):
                    forces[i, dim] = forces[i, dim] + force_mag * r_vec[dim] / r_mag
        
        self.force_calc_time = time.time() - start_time
        return forces
    
    def calculate_forces(self):
        """Calculate forces using selected precision mode."""
        if self.precision_mode == "balanced":
            return self.calculate_forces_balanced()
        else:
            return self.calculate_forces_standard()
        
    def update(self, num_steps=1):
        """Update the simulation for the given number of steps."""
        total_start = time.time()
        
        for _ in range(num_steps):
            # Calculate forces
            forces = self.calculate_forces()
            
            # Update velocities and positions
            for i, body in enumerate(self.bodies):
                if body.get("static", False):
                    continue
                
                # Update velocity (F = ma => a = F/m)
                self.velocities[i] += forces[i] / body["mass"] * self.TIME_STEP
                
                # Update position
                self.positions[i] += self.velocities[i] * self.TIME_STEP
                
                # Update body position
                body["position"] = self.positions[i]
                body["velocity"] = self.velocities[i]
                
                # Add to trail
                self.trails[i].append(body["position"].get().copy())
                
                # Keep trail length reasonable
                if len(self.trails[i]) > 8000:
                    self.trails[i].pop(0)
            
            # Update elapsed time
            self.elapsed_simulation_time += self.TIME_STEP
            
        self.total_calc_time = time.time() - total_start
        
    def get_positions(self):
        """Get current positions of all bodies."""
        return [body["position"].get() / self.AU for body in self.bodies]
        
    def get_trails(self):
        """Get all orbit trails in AU."""
        return [[np.array(pos) / self.AU for pos in trail] for trail in self.trails]
        
    def reset(self):
        """Reset the simulation to initial state."""
        self.init_celestial_bodies()
        self.elapsed_simulation_time = 0
        
    def get_performance_stats(self):
        """Get performance statistics."""
        return {
            "force_calc_time": self.force_calc_time * 1000,  # ms
            "total_calc_time": self.total_calc_time * 1000,  # ms
            "elapsed_days": self.elapsed_simulation_time / (24*3600),
            "elapsed_years": self.elapsed_simulation_time / (365.25*24*3600),
            "precision_mode": self.precision_mode
        }

def run_solar_system_simulation():
    """Run the solar system simulation with visualization."""
    import math
    
    # Initialize PyGame and OpenGL with improved settings
    pygame.init()
    display = (1600, 1200)
    
    # Use HWSURFACE, OPENGL, and enable vsync
    pygame.display.gl_set_attribute(pygame.GL_DOUBLEBUFFER, 1)
    pygame.display.gl_set_attribute(pygame.GL_SWAP_CONTROL, 1)  # Enable vsync
    screen = pygame.display.set_mode(display, DOUBLEBUF | OPENGL | pygame.HWSURFACE | pygame.FULLSCREEN)
    pygame.display.set_caption("Solar System N-Body Simulation - Tensor Core Edition")
    
    # Improved OpenGL setup
    glClearColor(0.0, 0.0, 0.1, 1.0)
    glEnable(GL_DEPTH_TEST)
    glShadeModel(GL_SMOOTH)
    glHint(GL_PERSPECTIVE_CORRECTION_HINT, GL_NICEST)
    
    # Projection setup
    glMatrixMode(GL_PROJECTION)
    gluPerspective(45, (display[0]/display[1]), 0.1, 100.0)
    glMatrixMode(GL_MODELVIEW)
    
    # Camera position
    camera_dist = 20.0
    camera_pitch = 30.0
    camera_yaw = 0.0
    
    # Create two simulators for comparison
    balanced_sim = BalancedTensorNBodySimulator(precision_mode="balanced")
    standard_sim = BalancedTensorNBodySimulator(precision_mode="standard")
    
    # Currently active simulator
    active_sim = balanced_sim
    
    # Simulation parameters
    paused = False
    time_scale = 5.0  # days per frame
    show_trails = True
    
    # Colors for bodies
    colors = {
        "Sun": (1.0, 1.0, 0.0),
        "Mercury": (0.7, 0.7, 0.7),
        "Venus": (0.8, 0.7, 0.5),
        "Earth": (0.0, 0.0, 1.0),
        "Mars": (1.0, 0.0, 0.0),
        "Jupiter": (0.8, 0.6, 0.0),
        "Saturn": (0.9, 0.8, 0.5),
        "Uranus": (0.5, 0.8, 0.9),
        "Neptune": (0.0, 0.0, 0.8)
    }
    
    # Font for text rendering
    pygame.font.init()
    font = pygame.font.SysFont('Arial', 18)
    
    # Create a separate surface for text overlay
    text_surface = pygame.Surface(display, pygame.SRCALPHA)
    
    # Main loop
    steps_per_frame = 5
    clock = pygame.time.Clock()
    
    running = True
    while running:
        # Handle events
        for event in pygame.event.get():
            if event.type == pygame.QUIT:
                running = False
                
            elif event.type == pygame.KEYDOWN:
                if event.key == pygame.K_ESCAPE:
                    running = False
                elif event.key == pygame.K_SPACE:
                    paused = not paused
                elif event.key == pygame.K_r:
                    balanced_sim.reset()
                    standard_sim.reset()
                elif event.key == pygame.K_t:
                    show_trails = not show_trails
                elif event.key == pygame.K_UP:
                    time_scale *= 1.5
                elif event.key == pygame.K_DOWN:
                    time_scale /= 1.5
                elif event.key == pygame.K_b:
                    active_sim = balanced_sim
                elif event.key == pygame.K_s:
                    active_sim = standard_sim
                    
            elif event.type == pygame.MOUSEMOTION:
                if pygame.mouse.get_pressed()[0]:
                    dx, dy = event.rel
                    camera_yaw += dx * 0.5
                    camera_pitch = max(-89, min(89, camera_pitch - dy * 0.5))
                    
            elif event.type == pygame.MOUSEBUTTONDOWN:
                if event.button == 4:  # Scroll up
                    camera_dist = max(5, camera_dist - 1)
                elif event.button == 5:  # Scroll down
                    camera_dist = min(50, camera_dist + 1)
        
        # Clear the color and depth buffers
        glClear(GL_COLOR_BUFFER_BIT | GL_DEPTH_BUFFER_BIT)
        
        # Update simulation
        if not paused:
            active_sim.TIME_STEP = 24*3600 * time_scale / steps_per_frame
            active_sim.update(steps_per_frame)
        
        # Reset model view matrix
        glLoadIdentity()
        
        # Set camera position with improved calculation
        gluLookAt(
            camera_dist * math.cos(math.radians(camera_yaw)) * math.cos(math.radians(camera_pitch)),
            camera_dist * math.sin(math.radians(camera_pitch)),
            camera_dist * math.sin(math.radians(camera_yaw)) * math.cos(math.radians(camera_pitch)),
            0, 0, 0,
            0, 1, 0
        )
        
        # Draw celestial bodies
        for i, body in enumerate(active_sim.bodies):
            glPushMatrix()
            
            pos = body["position"].get() / active_sim.AU
            glTranslatef(pos[0], pos[1], pos[2])
            
            # Set color
            color = colors.get(body["name"], (1, 1, 1))
            glColor3f(*color)
            
            # Draw sphere with improved quality
            size = body["size"] / 300.0  # Scale down for visualization
            quad = gluNewQuadric()
            gluQuadricNormals(quad, GLU_SMOOTH)
            gluQuadricTexture(quad, GL_TRUE)
            gluSphere(quad, size, 30, 30)  # Increased resolution
            gluDeleteQuadric(quad)
            
            glPopMatrix()
        
        # Draw orbit trails
        if show_trails:
            trails = active_sim.get_trails()
            
            glBegin(GL_LINES)
            for i, trail in enumerate(trails):
                if i == 0:  # Skip Sun
                    continue
                    
                color = colors.get(active_sim.bodies[i]["name"], (1, 1, 1))
                glColor3f(*color)
                
                for j in range(1, len(trail)):
                    glVertex3f(trail[j-1][0], trail[j-1][1], trail[j-1][2])
                    glVertex3f(trail[j][0], trail[j][1], trail[j][2])
            glEnd()
        
        # Render text overlay
        text_surface.fill((0, 0, 0, 0))  # Clear with transparent background
        
        # Render stats
        stats = active_sim.get_performance_stats()
        lines = [
            f"Precision Mode: {stats['precision_mode'].upper()}",
            f"Simulation Time: {stats['elapsed_years']:.2f} Earth years",
            f"Time Step: {time_scale:.1f} days/frame",
            f"Force Calculation: {stats['force_calc_time']:.2f} ms",
            "",
            "Controls:",
            "B: Balanced Precision",
            "S: Standard Precision",
            "Space: Pause/Resume",
            "R: Reset",
            "T: Toggle Trails",
            "Up/Down: Adjust Speed",
            "Mouse: Rotate View",
            "Scroll: Zoom In/Out"
        ]
        
        # Render text to overlay surface
        y = 20
        for line in lines:
            text_render = font.render(line, True, (255, 255, 255))
            text_surface.blit(text_render, (10, y))
            y += 20
        
        # Mode indicator
        mode_text = "BALANCED TENSOR CORES" if active_sim is balanced_sim else "STANDARD TENSOR CORES"
        mode_color = (0, 255, 0) if active_sim is balanced_sim else (255, 255, 0)
        mode_render = font.render(mode_text, True, mode_color)
        text_surface.blit(mode_render, (display[0] - mode_render.get_width() - 10, 20))
        
        # Swap buffers and render text overlay
        pygame.display.flip()
        
        # Control frame rate
        clock.tick(60)  # Cap at 60 FPS
    
    # Cleanup
    pygame.quit()

if __name__ == "__main__":
    run_solar_system_simulation()


Initializing CUDA...
CUDA initialization complete
Function signatures configured
Initializing CUDA...
CUDA initialization complete
Function signatures configured
